In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
paths="/content/drive/MyDrive/balance"
X=[]
Y=[]
Z=[]
for i in os.listdir(paths):
    file_path=os.path.join(paths,i)
    for x in os.listdir(file_path):
        file=os.path.join(file_path,x)

        if x=="train_GGNNinput_9444_balanced_graph.json":
            kq1=pd.read_json(file)
            X.append(kq1)
        if x=="test_GGNNinput_graph.json":
            kq2=pd.read_json(file)
            Y.append(kq2)
        if x=="valid_GGNNinput_graph.json":
            kq3=pd.read_json(file)
            Z.append(kq3)
# train=pd.concat(X ,ignore_index=True)
# valid=pd.concat(Z ,ignore_index=True)
# test=pd.concat(Y ,ignore_index=True)


In [ ]:
train=pd.concat(X,axis=0)
test=pd.concat(Y,axis=0)
valid=pd.concat(Z,axis=0)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)
train["graph_feature"]=train["graph_feature"].apply(lambda x : np.array(x))
test["graph_feature"]=test["graph_feature"].apply(lambda x : np.array(x))
valid["graph_feature"]=valid["graph_feature"].apply(lambda x : np.array(x))

In [ ]:
train["target"][0:10]

0    1
1    0
2    1
3    1
4    0
5    1
6    0
7    0
8    0
9    0
Name: target, dtype: int64

In [ ]:
train["graph_feature"]=train["graph_feature"].apply(lambda x : x.reshape(20,10))
test["graph_feature"]=test["graph_feature"].apply(lambda x : x.reshape(20,10))
valid["graph_feature"]=valid["graph_feature"].apply(lambda x : x.reshape(20,10))


In [ ]:
x_train=train["graph_feature"].to_numpy().tolist()
x_train=np.array(x_train)
x_test=test["graph_feature"].to_numpy().tolist()
x_test=np.array(x_test)
x_valid=valid["graph_feature"].to_numpy().tolist()
x_valid=np.array(x_valid)

In [ ]:
from keras.layers import LSTM, Dense,Dropout
from keras.models import Sequential

from sklearn.model_selection import train_test_split
import random
from keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(LSTM(units=256, return_sequences=True,input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=256, return_sequences=True))
model.add(Dropout(0.2))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(Dropout(0.2))
model.add(LSTM(units=256))
model.add(Dropout(0.2))
# model.add(LSTM(units=50))
# model.add(Dropout(0.2))
model.add(Dense(units= 1, activation="sigmoid"))

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)  # Tạo tối ưu hóa Adam với learning rate

model.compile(optimizer=optimizer, metrics=['accuracy'], loss="binary_crossentropy")

In [ ]:
model.fit(x_train,train["target"], epochs=3, batch_size=64)

Epoch 1/3
2952/2952 [==============================] - 1128s 379ms/step - loss: 0.4450 - accuracy: 0.7991
Epoch 2/3
2952/2952 [==============================] - 1157s 392ms/step - loss: 0.5861 - accuracy: 0.6952
Epoch 3/3
2952/2952 [==============================] - 1153s 391ms/step - loss: 0.5774 - accuracy: 0.7051


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
predicted_labels = model.predict(x_test)

1137/1137 [==============================] - 91s 79ms/step


In [ ]:
predicted_labels = (predicted_labels > 0.5).astype(int)

In [ ]:
accuracy = accuracy_score(test["target"], predicted_labels)
precision = precision_score(test["target"], predicted_labels)
recall = recall_score(test["target"], predicted_labels)
f1 = f1_score(test["target"], predicted_labels)

In [ ]:
print("accuracy: ",accuracy)
print("precision: ",precision)
print("recall: ",recall)
print("f1 :",f1)

accuracy:  0.6690869086908691
precision:  0.16992869063874982
recall:  0.6726726726726727
f1 : 0.2713178294573644
